<a href="https://colab.research.google.com/github/AbhinavCurseOK/Pest_crop/blob/main/pest_crop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
armaanoajay_ccmt_crop_pest_and_disease_detection_path = kagglehub.dataset_download('armaanoajay/ccmt-crop-pest-and-disease-detection')

print('Data source import complete.')


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense,Dropout,BatchNormalization

In [ ]:
import kagglehub
path = kagglehub.dataset_download("armaanoajay/ccmt-crop-pest-and-disease-detection")

In [ ]:
train_dir = "/kaggle/input/ccmt-crop-pest-and-disease-detection/split-new/train"
valid_dir = "/kaggle/input/ccmt-crop-pest-and-disease-detection/split-new/valid"
test_dir = "/kaggle/input/ccmt-crop-pest-and-disease-detection/split-new/test"
img_size = (224, 224)
batch_size = 64
num_classes = 22

In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=2,
    width_shift_range=0.025,
    height_shift_range=0.025,
    zoom_range=0.015,
    horizontal_flip=False
)
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)
valid_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    valid_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 15062 images belonging to 22 classes.
Found 5012 images belonging to 22 classes.
Found 5052 images belonging to 22 classes.


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
lr_scheduler = ReduceLROnPlateau(monitor='val_loss',
                                 factor=0.5,
                                 patience=3,
                                 min_lr=1e-6)

early_stopping = EarlyStopping(monitor='val_loss',
                               patience=5,
                               restore_best_weights=True)

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False
inputs = Input(shape=(224, 224, 3))
x = base_model(inputs)
x = GlobalAveragePooling2D()(x)
outputs = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 22)                  │          28,182 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,286,166 (8.72 MB)

 Trainable params: 28,182 (110.09 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
history = model.fit(train_generator,validation_data=valid_generator,epochs=10)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


236/236 ━━━━━━━━━━━━━━━━━━━━ 404s 2s/step - accuracy: 0.5021 - loss: 1.6259 - val_accuracy: 0.7420 - val_loss: 0.7404
Epoch 2/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 269s 1s/step - accuracy: 0.7713 - loss: 0.6617 - val_accuracy: 0.7664 - val_loss: 0.6584
Epoch 3/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 267s 1s/step - accuracy: 0.8066 - loss: 0.5539 - val_accuracy: 0.7841 - val_loss: 0.6014
Epoch 4/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 267s 1s/step - accuracy: 0.8196 - loss: 0.4954 - val_accuracy: 0.7899 - val_loss: 0.5760
Epoch 5/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 269s 1s/step - accuracy: 0.8422 - loss: 0.4515 - val_accuracy: 0.7923 - val_loss: 0.5654
Epoch 6/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 267s 1s/step - accuracy: 0.8516 - loss: 0.4228 - val_accuracy: 0.7997 - val_loss: 0.5574
Epoch 7/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 267s 1s/step - accuracy: 0.8540 - loss: 0.4135 - val_accuracy: 0.8017 - val_loss: 0.5398
Epoch 8/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 267s 1s/step - accuracy: 0.8600 - loss: 0.3878 - val_accuracy: 0.799

In [ ]:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name, "Trainable:", layer.trainable)

0 input_layer Trainable: False
1 Conv1 Trainable: False
2 bn_Conv1 Trainable: False
3 Conv1_relu Trainable: False
4 expanded_conv_depthwise Trainable: False
5 expanded_conv_depthwise_BN Trainable: False
6 expanded_conv_depthwise_relu Trainable: False
7 expanded_conv_project Trainable: False
8 expanded_conv_project_BN Trainable: False
9 block_1_expand Trainable: False
10 block_1_expand_BN Trainable: False
11 block_1_expand_relu Trainable: False
12 block_1_pad Trainable: False
13 block_1_depthwise Trainable: False
14 block_1_depthwise_BN Trainable: False
15 block_1_depthwise_relu Trainable: False
16 block_1_project Trainable: False
17 block_1_project_BN Trainable: False
18 block_2_expand Trainable: False
19 block_2_expand_BN Trainable: False
20 block_2_expand_relu Trainable: False
21 block_2_depthwise Trainable: False
22 block_2_depthwise_BN Trainable: False
23 block_2_depthwise_relu Trainable: False
24 block_2_project Trainable: False
25 block_2_project_BN Trainable: False
26 block_2_ad

In [ ]:
print(len(base_model.layers))

154


In [ ]:
base_model.trainable = True
for layer in base_model.layers[: -36]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(3e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history_finetune = model.fit(train_generator,
                             epochs=10,
                             validation_data=valid_generator,
                             callbacks=[early_stopping, lr_scheduler])

Epoch 1/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 289s 1s/step - accuracy: 0.7162 - loss: 0.8619 - val_accuracy: 0.7636 - val_loss: 0.7210 - learning_rate: 3.0000e-05
Epoch 2/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 268s 1s/step - accuracy: 0.8556 - loss: 0.3953 - val_accuracy: 0.7895 - val_loss: 0.6185 - learning_rate: 3.0000e-05
Epoch 3/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 266s 1s/step - accuracy: 0.8883 - loss: 0.3022 - val_accuracy: 0.8063 - val_loss: 0.5711 - learning_rate: 3.0000e-05
Epoch 4/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 267s 1s/step - accuracy: 0.9172 - loss: 0.2450 - val_accuracy: 0.8166 - val_loss: 0.5360 - learning_rate: 3.0000e-05
Epoch 5/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 267s 1s/step - accuracy: 0.9351 - loss: 0.2013 - val_accuracy: 0.8200 - val_loss: 0.5153 - learning_rate: 3.0000e-05
Epoch 6/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 267s 1s/step - accuracy: 0.9515 - loss: 0.1626 - val_accuracy: 0.8194 - val_loss: 0.5169 - learning_rate: 3.0000e-05
Epoch 7/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 269s 1s/step - acc

In [ ]:
print("Final Training Accuracy: {:.2f}%".format(history_finetune.history['accuracy'][-1] * 100))
print("Final Validation Accuracy: {:.2f}%".format(history_finetune.history['val_accuracy'][-1] * 100))

Final Training Accuracy: 97.48%
Final Validation Accuracy: 83.12%


In [ ]:
model.save("new_1_crop_disease_model.keras")

In [ ]:
from IPython.display import FileLink
FileLink(r'new_1_crop_disease_model.keras')

/kaggle/working/new_1_crop_disease_model.keras

In [ ]:
model = tf.keras.models.load_model("new_1_crop_disease_model.keras")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("crop_1_disease_model.tflite", "wb") as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmpn_oxb884'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 22), dtype=tf.float32, name=None)
Captures:
  136473613371696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136473613382608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136473613384720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136473613378032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136473613380320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136473613382080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136473612716336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136473612720912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136473612721088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136473612723200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136473612719

In [ ]:
from IPython.display import FileLink
FileLink(r'crop_1_disease_model.tflite')

/kaggle/working/crop_1_disease_model.tflite

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

In [ ]:
y_true = test_generator.classes
y_pred_probs = model.predict(test_generator)
y_pred = np.argmax(y_pred_probs, axis=1)
class_labels = list(test_generator.class_indices.keys())
print(classification_report(y_true, y_pred, target_names=class_labels))

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


79/79 ━━━━━━━━━━━━━━━━━━━━ 72s 917ms/step
                           precision    recall  f1-score   support

       anthracnose-cashew       0.88      0.84      0.86       347
 bacterial blight-cassava       0.91      0.85      0.88       524
       brown spot-cassava       0.84      0.90      0.87       297
    cashew-mosaic-cassava       0.93      0.95      0.94       241
      fall armyworm-maize       0.94      0.77      0.85        57
         grasshoper-maize       0.97      0.85      0.91       136
       green mite-cassava       0.92      0.88      0.90       203
           gumosis-cashew       0.99      0.96      0.97        79
           healthy-cashew       0.93      0.95      0.94       275
          healthy-cassava       0.91      0.97      0.94       240
            healthy-maize       0.79      0.76      0.77        41
           healthy-tomato       0.72      0.88      0.80        95
        leaf beetle-maize       0.89      0.96      0.93       189
        leaf blight

In [ ]:
print(class_labels)

['anthracnose-cashew', 'bacterial blight-cassava', 'brown spot-cassava', 'cashew-mosaic-cassava', 'fall armyworm-maize', 'grasshoper-maize', 'green mite-cassava', 'gumosis-cashew', 'healthy-cashew', 'healthy-cassava', 'healthy-maize', 'healthy-tomato', 'leaf beetle-maize', 'leaf blight-maize', 'leaf blight-tomato', 'leaf curl-tomato', 'leaf miner-cashew', 'leaf spot-maize', 'red rust-cashew', 'septoria leaf spot-tomato', 'streak virus-maize', 'verticulium wilt-tomato']


In [ ]:
model = load_model("new_1_crop_disease_model.keras")

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import numpy as np

y_true = test_generator.classes
y_pred_probs = model.predict(test_generator)

thresholds = np.arange(0.1, 1.0, 0.1)

print(f"{'Threshold':<10}{'Accuracy':<10}{'Precision':<10}{'Recall':<10}{'F1-score':<10}")
print(f"{'-'*50}")

for thresh in thresholds:
    y_pred = np.where(y_pred_probs >= thresh, 1, 0)
    y_pred_labels = np.argmax(y_pred, axis=1)
    accuracy = accuracy_score(y_true, y_pred_labels)
    precision = precision_score(y_true, y_pred_labels, average='weighted', zero_division=1)
    recall = recall_score(y_true, y_pred_labels, average='weighted', zero_division=1)
    f1 = f1_score(y_true, y_pred_labels, average='weighted', zero_division=1)
    print(f"{thresh:<10.2f}{accuracy:<10.4f}{precision:<10.4f}{recall:<10.4f}{f1:<10.4f}")

class_labels = list(test_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(y_true, np.argmax(y_pred_probs, axis=1), target_names=class_labels))

79/79 ━━━━━━━━━━━━━━━━━━━━ 36s 461ms/step
Threshold Accuracy  Precision Recall    F1-score  
--------------------------------------------------
0.10      0.8096    0.8220    0.8096    0.8035    
0.20      0.8316    0.8359    0.8316    0.8263    
0.30      0.8369    0.8372    0.8369    0.8321    
0.40      0.8369    0.8375    0.8369    0.8312    
0.50      0.8266    0.8323    0.8266    0.8210    
0.60      0.8064    0.8360    0.8064    0.8062    
0.70      0.7805    0.8431    0.7805    0.7883    
0.80      0.7466    0.8541    0.7466    0.7645    
0.90      0.6902    0.8735    0.6902    0.7245    

Classification Report:
                           precision    recall  f1-score   support

       anthracnose-cashew       0.88      0.84      0.86       347
 bacterial blight-cassava       0.91      0.85      0.88       524
       brown spot-cassava       0.84      0.90      0.87       297
    cashew-mosaic-cassava       0.93      0.95      0.94       241
      fall armyworm-maize       0.94 

In [ ]:
y_true = test_generator.classes
y_pred_probs = model.predict(test_generator)

thresholds = np.arange(0.4, 1.0, 0.05)

print(f"{'Threshold':<10}{'Accuracy':<10}{'Precision':<10}{'Recall':<10}{'F1-score':<10}")
print(f"{'-'*50}")

for thresh in thresholds:
    y_pred = np.where(y_pred_probs >= thresh, 1, 0)
    y_pred_labels = np.argmax(y_pred, axis=1)
    accuracy = accuracy_score(y_true, y_pred_labels)
    precision = precision_score(y_true, y_pred_labels, average='weighted', zero_division=1)
    recall = recall_score(y_true, y_pred_labels, average='weighted', zero_division=1)
    f1 = f1_score(y_true, y_pred_labels, average='weighted', zero_division=1)
    print(f"{thresh:<10.2f}{accuracy:<10.4f}{precision:<10.4f}{recall:<10.4f}{f1:<10.4f}")

class_labels = list(test_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(y_true, np.argmax(y_pred_probs, axis=1), target_names=class_labels))

79/79 ━━━━━━━━━━━━━━━━━━━━ 45s 542ms/step
Threshold Accuracy  Precision Recall    F1-score  
--------------------------------------------------
0.40      0.8369    0.8375    0.8369    0.8312    
0.45      0.8325    0.8334    0.8325    0.8264    
0.50      0.8266    0.8323    0.8266    0.8210    
0.55      0.8181    0.8331    0.8181    0.8145    
0.60      0.8064    0.8360    0.8064    0.8062    
0.65      0.7947    0.8383    0.7947    0.7977    
0.70      0.7805    0.8431    0.7805    0.7883    
0.75      0.7662    0.8499    0.7662    0.7789    
0.80      0.7466    0.8541    0.7466    0.7645    
0.85      0.7221    0.8603    0.7221    0.7477    
0.90      0.6902    0.8735    0.6902    0.7245    
0.95      0.6285    0.8873    0.6285    0.6752    

Classification Report:
                           precision    recall  f1-score   support

       anthracnose-cashew       0.88      0.84      0.86       347
 bacterial blight-cassava       0.91      0.85      0.88       524
       brown spot-

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import numpy as np

y_true = test_generator.classes
y_pred_probs = model.predict(test_generator)

threshold = 0.395  #.435

y_pred = np.where(y_pred_probs >= threshold, 1, 0)
y_pred_labels = np.argmax(y_pred, axis=1)

accuracy = accuracy_score(y_true, y_pred_labels)
precision = precision_score(y_true, y_pred_labels, average='weighted', zero_division=1)
recall = recall_score(y_true, y_pred_labels, average='weighted', zero_division=1)
f1 = f1_score(y_true, y_pred_labels, average='weighted', zero_division=1)

print(f"{'Metric':<12}{'Value':<10}")
print(f"{'-'*22}")
print(f"{'Accuracy':<12}{accuracy:<10.4f}")
print(f"{'Precision':<12}{precision:<10.4f}")
print(f"{'Recall':<12}{recall:<10.4f}")
print(f"{'F1-score':<12}{f1:<10.4f}")

class_labels = list(test_generator.class_indices.keys())
print("\nClassification Report:")
print(classification_report(y_true, np.argmax(y_pred_probs, axis=1), target_names=class_labels))

79/79 ━━━━━━━━━━━━━━━━━━━━ 37s 462ms/step
Metric      Value     
----------------------
Accuracy    0.8373    
Precision   0.8379    
Recall      0.8373    
F1-score    0.8318    

Classification Report:
                           precision    recall  f1-score   support

       anthracnose-cashew       0.88      0.84      0.86       347
 bacterial blight-cassava       0.91      0.85      0.88       524
       brown spot-cassava       0.84      0.90      0.87       297
    cashew-mosaic-cassava       0.93      0.95      0.94       241
      fall armyworm-maize       0.94      0.77      0.85        57
         grasshoper-maize       0.97      0.85      0.91       136
       green mite-cassava       0.92      0.88      0.90       203
           gumosis-cashew       0.99      0.96      0.97        79
           healthy-cashew       0.93      0.95      0.94       275
          healthy-cassava       0.91      0.97      0.94       240
            healthy-maize       0.79      0.76      0.77  

In [ ]:
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2

model = load_model("new_1_crop_disease_model.keras")

x = model.layers[-2].output
x = Dense(768, activation="relu", kernel_regularizer=l2(1e-4))(x)
x = Dropout(0.4)(x)
x = Dense(256, activation="relu", kernel_regularizer=l2(1e-4))(x)
x = Dropout(0.25)(x)
output = Dense(22, activation="softmax")(x)
new_model = Model(inputs=model.input, outputs=output)

for layer in model.layers[:-5]:
    layer.trainable = False
new_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])